In [ ]:
# !pip install ultralytics

In [2]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from ultralytics import YOLO

In [3]:
class YOLOSegmentation:
    def __init__(self, model_path):
        self.model = YOLO(model_path)

    def detect(self, img):

        height, width, channels = img.shape

        results = self.model.predict(source=img.copy(), save=False, save_txt=False)
        result = results[0]

        segmentation_contours_idx = []

        for seg in result.masks.segments:
            seg[:, 0] *= width
            seg[:, 1] *= height
            segment = np.array(seg, dtype=np.int32)
            segmentation_contours_idx.append(segment)

        bboxes = np.array(result.boxes.xyxy.cpu(), dtype="int")
        class_ids = np.array(result.boxes.cls.cpu(), dtype="int")
        scores = np.array(result.boxes.conf.cpu(), dtype="float")

        return bboxes, class_ids, segmentation_contours_idx, scores

In [ ]:
img = cv2.imread("rugby_ball.jpeg")
cv2_imshow(img)

In [5]:
img = cv2.resize(img, None, fx=0.7, fy=0.7)

In [6]:
ys = YOLOSegmentation("yolov8m-seg.pt")

In [ ]:
bboxes, classes, segmentations, scores = ys.detect(img)
for bbox, class_id, seg, score in zip(bboxes, classes, segmentations, scores):
    # print("bbox:", bbox, "class id:", class_id, "seg:", seg, "score:", score)
    (x, y, x2, y2) = bbox

    print(seg)

In [9]:
cv2.rectangle(img, (x, y), (x2, y2), (255, 0, 0), 2)
cv2_imshow(cv2.polylines(img, [seg], True, (0, 0, 255), 4))

Output hidden; open in https://colab.research.google.com to view.